In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import Imputer,StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing

In [2]:
x_train = pd.read_csv('../data/pre_x_train.csv')
del x_train['id']
del x_train['timestamp']
x_train_id = pd.read_csv('../data/pre_x_train.csv',usecols=['id'])
x_train_time = pd.read_csv('../data/pre_x_train.csv',usecols=['timestamp'])
y_train = pd.read_csv('../data/pre_y_train.csv')

x_test = pd.read_csv('../data/pre_x_test.csv')
del x_test['id']
del x_test['timestamp']
x_test_id = pd.read_csv('../data/pre_x_test.csv',usecols=['id'])
x_test_time = pd.read_csv('../data/pre_x_test.csv',usecols=['timestamp'])

macro = pd.read_csv('../data/pre_macro.csv')

In [3]:
for f in x_train.columns:
    if x_train[f].dtype==object:
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_test[f].values)+list(x_train[f].values)) 
        x_train[f] = lbl.transform(list(x_train[f].values))
        x_test[f] = lbl.transform(list(x_test[f].values))

In [4]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.3, random_state=4141)

In [6]:
params = {}
params['objective'] = 'reg:linear'
params['eval_metric'] = 'rmse'
params['eta'] = 0.02
params['max_depth'] = 5

In [7]:
d_train = xgb.DMatrix(x_train,y_train)
d_valid = xgb.DMatrix(x_valid,y_valid)
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
bst = xgb.train(params, d_train, 1000, watchlist, early_stopping_rounds=50, verbose_eval=10)

[0]	train-rmse:14.8174	valid-rmse:14.8247
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 50 rounds.
[10]	train-rmse:12.1113	valid-rmse:12.1182
[20]	train-rmse:9.9009	valid-rmse:9.90756
[30]	train-rmse:8.09575	valid-rmse:8.10224
[40]	train-rmse:6.62187	valid-rmse:6.62817
[50]	train-rmse:5.4189	valid-rmse:5.42507
[60]	train-rmse:4.43751	valid-rmse:4.44355
[70]	train-rmse:3.63747	valid-rmse:3.64337
[80]	train-rmse:2.98592	valid-rmse:2.99191
[90]	train-rmse:2.45612	valid-rmse:2.46219
[100]	train-rmse:2.02628	valid-rmse:2.03227
[110]	train-rmse:1.6787	valid-rmse:1.6846
[120]	train-rmse:1.39892	valid-rmse:1.40475
[130]	train-rmse:1.17518	valid-rmse:1.18096
[140]	train-rmse:0.997786	valid-rmse:1.00357
[150]	train-rmse:0.858675	valid-rmse:0.864595
[160]	train-rmse:0.751088	valid-rmse:0.757024
[170]	train-rmse:0.669065	valid-rmse:0.675011
[180]	train-rmse:0.607968	valid-rmse:0.613886
[190]	train-rmse:0.562887

In [ ]:
fig, ax = plt.subplots(figsize=(12,18))
xgb.plot_importance(bst, max_num_features=50, height=0.8, ax=ax)
plt.show()

In [ ]:
d_test = xgb.DMatrix(x_test)
y_test = bst.predict(d_test)

In [ ]:
sub = pd.DataFrame()
sub['id'] = np.array(x_test_id['id'].values,np.int32)
sub['price_doc'] = np.exp(y_test)-1.0
sub.to_csv('../output/simple5.csv', index=False)